In [ ]:
#Data Science
#Coder: Marco Galicia
#Alma mater: IPN Mexico - ESIME ICE
#Seminario:Patrones Area Médica 2019
#Work based on:
#Hands on ML - Aurelien Geron - O'reilly media Inc,
#https://www.kaggle.com/sarques/aiwitbor/notebook
#https://kapernikov.com/tutorial-image-classification-with-scikit-learn/
#https://becominghuman.ai/understanding-the-basics-of-cnn-with-image-classification-7f3a9ddea8f9
#"Yo solo sé que no se nada"...olvida todas las palabras griegas y los rollos de papiro...
import numpy as np # linear algebra
from numpy import array
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

print(os.listdir("../input/skin-cancer-mnist-ham10000"))
base_add = os.path.join('..', 'input/skin-cancer-mnist-ham10000')
print(base_add)
# Any results you write to the current directory are saved as output.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import sklearn.linear_model
from glob import glob
%matplotlib inline
from PIL import Image
from skimage import io, color, data
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import label_binarize, StandardScaler
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.linear_model import SGDClassifier

In [ ]:
#https://stackoverflow.com/questions/8532929/two-asterisks-in-file-path
#https://docs.python.org/2/library/glob.html
#https://www.geeksforgeeks.org/python-os-path-splitext-method/
#dark side of the force, Sith native language... x : x for x
image_path_dict = {os.path.splitext(os.path.basename(x))[0]: 
                   x for x in glob(os.path.join(base_add, '*', '*.jpg'))}
#https://stackoverflow.com/questions/9221817/python-dictionary
lesion_type_dict = {'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'}
df = pd.read_csv(os.path.join(base_add, 'HAM10000_metadata.csv'))
#df = pd.read_csv(os.path.join(base_add, 'HAM10000_metadata.csv'),nrows = 4000)
df.sort_values(by=["image_id"], ascending=False)
#tipo
print(type(df))
#longitud
print(len(df))

In [ ]:
#mapeo dinamico, creacion de nuevas series cell_type, path y dx_code
#operacion en bruto, se observa el comportamiento de series.map...
#si en df['dx'].iloc[0] = 'bkl', map busca y encuentra 'Benign keratosis-like lesions'
#si en df['dx'].iloc[10010] = 'akeic', map busca y encuentra 'Actinic keratoses'
#busqueda y asignacion com map key:value
df['cell_type'] = df['dx'].map(lesion_type_dict.get)
df['path'] = df['image_id'].map(image_path_dict.get)
#series(columnas) del data frame despues de la asignacion
print(df.columns)
#df.sample(3)
df.head(10)

In [ ]:
#analisis datos para preparar entrenamiento supervisado
#crea categorias para (dx): 7 to many, nueva serie/columna dx_code
df['dx_code'] = pd.Categorical(df['dx']).codes
print(pd.Categorical(df['dx']).unique())
print(pd.Categorical(df['dx_code']).unique())
#confirmando...
#bkl=2; nv=5; df=3; mel=4; vasc=6; bcc=1; akeic=0;
#df.sample(7)
df.head(10)

In [ ]:
#homogeneizar
df['age'].fillna(df['age'].mean(), inplace = True)
df.info()

In [ ]:
#https://stackoverflow.com/questions/43027980/purpose-of-matplotlib-inline
df['cell_type'].value_counts().plot(kind = 'bar')
print(df['cell_type'].value_counts())

In [ ]:
df['image_3d'] = df['path'].map(lambda x: np.asarray(Image.open(x).resize((100, 75))))
#df['image_2d'] = df['image_3d'].map(lambda x: rgb2gray(x.reshape(75,100,3)))
df['image_2d'] = df['image_3d'].map(lambda x: rgb2gray(x))
df = df.assign(image_1d=np.random.randn(10015))
print(df.columns)
imageSeriesSize = df['image_3d'].size
for i in range(imageSeriesSize):
    d3_sum = 0
    d2_sum = 0
    d1_sum = 0
    d1_mean = 0
    d3_sum = df['image_3d'].iloc[i]
    d2_sum = d3_sum.sum(axis=0)
    d1_sum = d2_sum.sum(axis=0)
    d1_mean = d1_sum.mean(axis=0)
    df.assign(image_1d=d1_mean)
#Reduccion aun numero, No hay imagenes repetidas
print(df['image_1d'].is_unique)
#analisis para un elemento de la serie path
#serieElementat = df['path'].iloc[2000]
#oneElement = np.asarray(Image.open(serieElementat).resize((100, 75)))
#print(oneElement.min())
#print(oneElement.max())
#print(df.columns)
#df.sample(2)
df.head(3)

In [ ]:
#Confirmando la categorizacion por tipo de lesion
# valores unico en la serie dx_code (etiquetas-label)
labels = np.unique(df['dx_code'])
 
# set up the matplotlib figure and axes, based on the number of labels RGB & GRAYscale
fig, axes = plt.subplots(1, len(labels))
fig.set_size_inches(15,4)
fig.tight_layout()

figGR, axesGR = plt.subplots(1, len(labels))
figGR.set_size_inches(15,4)
figGR.tight_layout()
          
#https://www.digitalocean.com/community/tutorials/how-to-use-break-continue-and-pass-statements-when-working-with-loops-in-python-3
for axRGB, dx_code in zip(axes, labels):
    #print(dx_code)
    for j in range(len(df['image_3d'])):
        if df['dx_code'].iloc[j] == dx_code:
            #print("found:",df['dx_code'].iloc[j])
            axRGB.imshow(df['image_3d'].iloc[j])
            axRGB.axis('off')
            axRGB.set_title(dx_code)
            break

for axGR, dx_code in zip(axesGR, labels):
    #print(dx_code)
    for k in range(len(df['image_2d'])):
        if df['dx_code'].iloc[k] == dx_code:
            #print("found:",df['dx_code'].iloc[j])
            axGR.imshow(df['image_2d'].iloc[k], cmap=plt.cm.gray)
            axGR.axis('off')
            axGR.set_title(dx_code)
            break

Se comparan dos algoritmos para clasificacion , Stochastic Gardient Descent (SGDCLassifier/sklearn) y Convolutional Neural Network (Conv2D/Keras).

Para utilizar los datos en SGDCLassifier, se necesita un preprocesamiento extra, el formato y forma del arreglo:

In [ ]:
X_preGray, y_gray = df['image_2d'], df['dx_code']
print("X_preGray shape:", X_preGray.shape)
print("X_preGray size:", X_preGray.size)
print("X_preGray inner shape:", X_preGray.iloc[0].size)
print("X_preGray type",type(X_preGray))
###################
#extraccion de nested array
aux_arr = []
for i in range(X_preGray.size):
    #aplastar el arreglo 75x100 para obtener lista 7500
    aux_arr.append(X_preGray.iloc[i].ravel())
    
X_gray = np.asarray(aux_arr)
#formato correcto para SGDClassifier (numero de imagenes, numero de features)
print("Forma esperada X_gray shape", X_gray.shape)
print(X_preGray)
print(X_gray)

A continuacion se presenta el algoritmo SGD sobre HAM1000 con las imagenes transformadas a GrayScale

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_gray,
    y_gray,
    test_size=0.3,
    shuffle=True,
    random_state=42,
)

In [ ]:
#binary classifier set up
#SGDClassifier
sgd_clf = SGDClassifier(random_state=42, max_iter=1000, tol=1e-3, alpha=0.00020)
sgd_clf.fit(X_train, y_train)
y_pred_test = sgd_clf.predict(X_test)

In [ ]:
print("...", type(y_pred_test))
print("...", type(y_test))
print("...", len(y_test))
print(y_pred_test)
print(y_test)
print('Porcentaje de precision en la prediccion: ')
print(100*np.sum(y_pred_test == y_test)/len(y_test))

In [ ]:
#analisis de error
y_test_pred = cross_val_predict(sgd_clf, X_test, y_test, cv=7)
print("conteo de validacion")
print(pd.value_counts(y_test))
conf_mx = confusion_matrix(y_test, y_test_pred)
print("matriz de confusion")
print(conf_mx)

In [ ]:
plt.matshow(conf_mx, cmap=plt.cm.gray)
plt.show()

Inferencia: Grafica estatica, la diagonal de la matriz de confusion vs los datos de validacion

In [ ]:
#version estatica
import matplotlib
import matplotlib.pyplot as plt
import numpy as np


labels = ['akeic', 'bcc', 'bkl', 'df', 'mel', 'nv' , 'vasc']
validacion = [96, 143, 350, 39, 347 , 1997, 33 ]
inferencia = [9, 34, 11, 1, 137, 1374, 1]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, validacion, width, label='Conteo Validación')
rects2 = ax.bar(x + width/2, inferencia, width, label='Conteo inferencia')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Conteo de lesiones')
ax.set_title('Conteo autómata SGDClassifier ≈ 66.8%')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)

fig.tight_layout()

plt.show()